Dataset is downloaded from Kaggle. Link: https://www.kaggle.com/giripujar/hr-analytics

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../input/hr-analytics/HR_comma_sep.csv')
df.head()

### Preliminary data exploration

In [ ]:
df[df['left']==1].shape[0] # No of employees lost by the company

In [ ]:
df[df['left']==0].shape[0] # No of employees retained by the company

In [ ]:
df.groupby('left').mean() 

Based on the above observation:
 Parameters- satisfaction_level, average_monthly_hours and promotion_last_5years seem to have the most effect on retention rate

### Employee retention salary-wise: Stacked Bar Chart

In [ ]:
df2 = df[['left','salary']]
left = df2[df['left']==1].salary.value_counts()
retained = df2[df['left']==0].salary.value_counts()
counts = {"retained":retained, "left":left}
counts = pd.DataFrame(counts)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def stacked_chart(df, title):
    fields = df.columns.tolist()
    category = df.index.tolist()
    ypos = np.arange(len(category))
    colors = ['#1D2F6F', '#8390FA']
    
    print(df)
    
    # Stacked Bar Chart
    # figure and axis
    fig, ax = plt.subplots(1, figsize=(12, 10))

    left = len(df)*[0]
    for idx, name in enumerate(fields):
        plt.barh(df.index,df[name],left=left, color=colors[idx])
        left = left + df[name]
    
    # title, legend, labels
    plt.title('Employee retention by salary', loc='left')
    plt.legend(fields, bbox_to_anchor=([0.55, 1, 0, 0]), ncol=4, frameon=False)
    plt.xlabel('No of Employees')
    
    # Stacked Percentage Bar Chart
    
    df['Total'] = df[fields].sum(axis=1)
    idx = len(df.columns)
    # create a column for each salary segment proportion of total
    for i in fields:
        df['{}_Percent'.format(i)] = df[i] / df['Total']


    df_p = df[df.columns.tolist()[idx:]]
    
    title = 'Employee retention percentage salary-wise'
    fields = df_p.columns.tolist() 
    # figure and axis
    fig, ax = plt.subplots(1, figsize=(12, 10))

    # plot bars
    left = len(df_p) * [0]
    for idx, name in enumerate(fields):
        plt.barh(df_p.index, df_p[name], left = left, color=colors[idx])
        left = left + df_p[name]
        
    # title
    plt.title(title, loc='left')
    
    # format x ticks
    xticks = np.arange(0,1.1,0.1)
    xlabels = ['{}%'.format(i) for i in np.arange(0,101,10)]
    plt.xticks(xticks, xlabels)
    plt.show()


In [ ]:
stacked_chart(counts,"Employee retention: Salary-wise")

The above graph shows that attrition rate is higher in lower salary level.

### Department-wise employee retention rate

In [ ]:
df3 = df[['left','Department']]

In [ ]:
depts = df3.Department.unique().tolist()

In [ ]:
left = df3[df3['left']==1].Department.value_counts()
retained = df3[df3['left']==0].Department.value_counts()
dept_counts = pd.DataFrame({"retained":retained, "left":left})
dept_counts

In [ ]:
stacked_chart(dept_counts,"Employee retention: department-wise")

### Logistics Regression model

In [ ]:
df1 = df[['salary','satisfaction_level', 'average_montly_hours', 'promotion_last_5years','left']]

In [ ]:
df1

In [ ]:
dummies = pd.get_dummies(df1.salary)
dummies

In [ ]:
df1 = pd.concat([df1,dummies],axis = 'columns')

In [ ]:
df1 = df1.drop(['salary','medium'],axis='columns')
df1

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(df1[['satisfaction_level', 'average_montly_hours', 'promotion_last_5years','high','low']],df1.left, test_size = 0.001,random_state = 1)

In [ ]:
X_test

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)